In [13]:
# Base ------------------------
import numpy as np
import pandas as pd
import scipy.stats as ss

# Inference -------------------
import statsmodels.stats.weightstats as smw
from statsmodels.stats.power import TTestPower

In [2]:
from google.colab import drive
drive.mount('IEXL')

Mounted at IEXL


In [3]:
anorexia = pd.read_excel('/content/IEXL/MyDrive/IEXL - Bootcamp - Math&Stats/Data Sets/anorexia.xlsx')
anorexia.head()

,ID,group,prewt,postwt,difwt
0,101,1,80.5,82.2,1.7
1,201,2,80.7,80.2,-0.5
2,301,3,83.8,95.2,11.4
3,102,1,84.9,85.6,0.7
4,202,2,89.4,80.1,-9.3


**Find the 99% confidence interval of the difference in weight for all the treatments. Interpret it**

In [6]:
# Data ------------------------------------
n = anorexia.shape[0]
SL = 0.01

# Descritipve Stats object ----------------
des = smw.DescrStatsW(anorexia['difwt'])

# Confidence Interval ---------------------
if n > 40:
  low, up = des.zconfint_mean(SL)
else:
  low, up = des.tconfint_mean(SL)

print(f'The {1-SL:4.0%} CI for the average difference in weight is [{low:4.2f}, {up:4.2f}]')

The  99% CI for the average difference in weight is [0.34, 5.19]


Let's now split in groups of treatments and see the 99% confidence interval again

In [5]:
groups = anorexia['group'].unique()

for i in groups:
  # Descritipve Stats object ----------------
  des = smw.DescrStatsW(anorexia[anorexia['group'] == i]['difwt'])
  n = anorexia[anorexia['group'] == i].shape[0]

  # Confidence Interval ---------------------
  if n > 40:
    low, up = des.zconfint_mean(SL)
  else:
    low, up = des.tconfint_mean(SL)

  print(f'The {1-SL:4.0%} CI for the average difference weight in group {i} is [{low:4.2f}, {up:4.2f}]')

The  99% CI for the average difference weight in group 1 is [-0.74, 6.76]
The  99% CI for the average difference weight in group 2 is [-4.82, 3.92]
The  99% CI for the average difference weight in group 3 is [2.19, 12.33]


**Let's test for a 5% of significance level if the difference in weight after the treatment in the whole sample was larger than 0**

The decision scheme is

\begin{equation}
H_0:\{\mu\leq 0\},\quad H_1:\{\mu > 0\}
\end{equation}

In [12]:
# Data ----------------------------------
mu0 = 0
SL = 0.05

# Test ----------------------------------
des = smw.DescrStatsW(anorexia['difwt'])

if n > 40:
  zstat, pval = des.ztest_mean(mu0, alternative = 'larger')
else:
  tstat, pval, dof = des.ttest_mean(mu0, alternative = 'larger')

print(f'alpha: {SL:4.2%}')
print(f'p-value: {pval:4.2%}')
if pval < SL:
  print('Reject H0')
else:
  print('Fail to Reject H0')

alpha: 5.00%
p-value: 0.17%
Reject H0


Since we reject $H_0$, that means that we have found evidence in favor of the alternative. Then the conclusion is that, for a 5% of significance level, we can say that the average weight after the treatments is significantly larger than 0.

**Suppose that you have read in a published peer-reviewed paper, that with these treatments, the average gained weight is of only 1 pound. Find the probability that if this was actually the case, you may detect if from your sample**

In [35]:
power = TTestPower()

now the decision scheme is

\begin{equation}
H_0:\{\mu = 0\},\quad H_1:\{\mu = 1\}
\end{equation}

In [19]:
# Data ---------------------------
SL = 0.05
mu0 = 0
mu1 = 1
std = anorexia['difwt'].std()
n = anorexia['difwt'].shape[0]

power.solve_power(effect_size = (mu0-mu1)/std,
                  nobs = n,
                  alpha = SL,
                  alternative = 'smaller')

0.2768799823824906

In the case of a right-tailed test, the power

\begin{equation}
\text{power} = P\left(Z > \frac{\mu_0-\mu_1}{s/\sqrt{n}} + z_\alpha\right)
\end{equation}

In [33]:
# Values ----------------------------------
zcrit = ss.norm.isf(SL)
effect_size = (mu0-mu1)/(std/np.sqrt(n))
zval = zcrit + effect_size

# Power -----------------------------------
power = ss.norm().sf(zval)
print(power)

0.2802785137165221


**Suppose that you want that probability to be of $95\%$, determine the sample size needed in this case**

In [36]:
pwr = 0.95

power.solve_power(effect_size = (mu0-mu1)/std,
                  alpha = SL,
                  power = pwr,
                  alternative = 'smaller')

691.1371750371576

Then the sample size needed to have a probability of 95% of detecting an average increase of 1 pound after all the treatments if this change actually happens is of 692 subjects.

**Test if for all the groups (separatedly) the average weight after the treatment is increased and then find the probability that if the increase (or decrease in the corresponding cases) is of one unit, you may detect it from your sample**

In [54]:
SL = 0.05
mu0 = 0
mu1 = 1

groups = anorexia['group'].unique()

for i in groups:
  std = anorexia[anorexia['group'] == i]['difwt'].std()
  # Descritipve Stats object ----------------
  des = smw.DescrStatsW(anorexia[anorexia['group'] == i]['difwt'])
  n = anorexia[anorexia['group'] == i]['difwt'].shape[0]

  print(f'\nGROUP {i}')
  print('-'*10)

  if n > 40:
    print('\nUsing the Normal Approximation...')
    zstat, pval = des.ztest_mean(mu0, alternative = 'larger')
  else:
    print('\nUsing the t-Distribution...')
    tstat, pval, dof = des.ttest_mean(mu0, alternative = 'larger')

  pwr = power.solve_power(effect_size = (mu0-mu1)/std,
                  nobs = n,
                  alpha = SL,
                  alternative = 'smaller')
  
  print(f'\nalpha: {SL:4.2%}')
  print(f'p-value: {pval:4.2%}')
  if pval < SL:
    print('Reject H0')
  else:
    print('Fail to Reject H0')

  print(f'\nPower of the test: {pwr:4.2%}')


GROUP 1
----------

Using the t-Distribution...

alpha: 5.00%
p-value: 1.75%
Reject H0

Power of the test: 17.73%

GROUP 2
----------

Using the t-Distribution...

alpha: 5.00%
p-value: 61.18%
Fail to Reject H0

Power of the test: 15.30%

GROUP 3
----------

Using the t-Distribution...

alpha: 5.00%
p-value: 0.04%
Reject H0

Power of the test: 13.72%
